In [12]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import dill
from pytictoc import TicToc
from IPython.display import clear_output
from cec2013lsgo.cec2013 import Benchmark

In [13]:
def DSRegPSO(D,n,Ll,Lu,c1,c2,Mmax,lambd,fdmax,fdmin,Smax,Smin,dseta,fevalmax,run, oparam,function,f):
    LSui = lambd*(Lu-Ll)
    LSu = LSui
    LSl = -LSu
    w = Mmax/2
    X = np.random.uniform(Ll,Lu,(n,D))
    V = np.zeros(X.shape)
    C = np.array([f(X[i,:])for i in range(X.shape[0])])
    feval = n
    GPi = C
    Pi = X
    GBid = np.argmin(GPi)
    GB = GPi[GBid]
    PG = X[GBid]
    GB0 = GB
    Ss = Smin
    deltamax = np.max(np.abs(PG))*fdmax
    deltamin = np.min(np.abs(PG))*fdmin
    delta = deltamin
    it = 0
    GBlist = [[]]*int(fevalmax/n)
    GBlist[it] = GB 
    t = TicToc()
    t.tic()
    while feval+n <= fevalmax:
        R1 = np.random.random(X.shape)
        R2 = np.random.random(X.shape)
        V = ((delta/deltamax)+(Ss/Smax))*w*V+c1*R1*(PG-X)+c2*R2*(Pi-X)
        V = np.clip(V, LSl,LSu)
        deltai = np.sum(np.absolute(X-PG),axis = 1)
        U = np.repeat((deltai < delta).reshape(n,1),D,axis = 1)
        psi = np.random.uniform(Ll,Lu,(n,D))
        X = (1-U)*(X+V)+U*psi
        X[X>Lu] = np.maximum(Ll,Lu+(Lu-X[X>Lu]))
        X[X<Ll] = np.minimum(Lu,Ll+Ll-X[X<Ll])
        X = np.clip(X, Ll,Lu)
        C = np.array([f(X[i,:])for i in range(X.shape[0])])
        feval += n
        for i in range(n):
            if C[i]<GPi[i]:
                GPi[i]=C[i]
                Pi[i,:] = X[i,:]
                if GPi[i]<GB:
                    GB = GPi[i]
                    PG = Pi[i,:]
        it+=1
        
        if GB == 0:
            print('Function',function,'Iteration',f"{it:.4E}",'Function Evaluations',f"{(feval):.4E}", 'Best Cost', f"{GB:.4E}", 'Time', time)
            print('Run',run,'Parameter',oparam)
            print('n',n)
            print('dseta',dseta)
            print('fdmax',fdmax)
            print('fdmin',fdmin)
            print('LSu',LSu)
            print('delta',delta)
            print('deltamin',deltamin)
            print('deltamax',deltamax)
            print('Ss',Ss)
            return GBlist,GB,PG
        if np.absolute(GB-GB0)<=dseta*GB:
            if delta<deltamax:
                delta = delta+deltamax*Ss
            else:
                delta = deltamin
                if Ss>=Smax:
                    Ss = Smin
                else:
                    Ss = Ss+Smin        
        else:
            delta = deltamin
            Ss = Smin
        deltamax = np.max(np.abs(PG))*fdmax
        deltamin = np.min(np.abs(PG))*fdmin
        LSu = (((delta/deltamax)+(Ss/Smax))/2)*(LSui)
        LSl = -LSu
        GB0 = GB
        GBlist[it] = GB 
        if np.mod(it,1000)==0:
            time = t.tocvalue()
            clear_output(wait=True)
            print('Function',function, 'Iteration',f"{it:.4E}",'Function Evaluations',f"{feval:.4E}", 'Best Cost', f"{GB:.4E}", 'Time', time)
            print('Run',run,'Parameter',oparam)
            print('n',n)
            print('dseta',dseta)
            print('fdmax',fdmax)
            print('fdmin',fdmin)
            print('LSu',LSu)
            print('delta',delta)
            print('deltamin',deltamin)
            print('deltamax',deltamax)
            print('Ss',Ss)
            t.tic()
    return GBlist,GB,PG

In [14]:
if __name__ == '__main__':
    DATA = pd.DataFrame({'alg':['DSRegPSOP']*3,
              'milestone':[1.2e5,6.0e5,3.0e6],
                'f1':[float('inf')]*3,'f2':[float('inf')]*3,'f3':[float('inf')]*3,'f4':[float('inf')]*3,'f5':[float('inf')]*3,
                'f6':[float('inf')]*3,'f7':[float('inf')]*3,'f8':[float('inf')]*3,'f9':[float('inf')]*3,
                'f10':[float('inf')]*3,'f11':[float('inf')]*3,'f12':[float('inf')]*3,'f13':[float('inf')]*3,'f14':[float('inf')]*3,
                'f15':[float('inf')]*3})
    DATA2 = pd.DataFrame({'Function':['f1','f2','f3','f4','f5','f6','f7','f8','f9','f10','f11','f12','f13','f14','f15'],
                          'Median':[[]]*15,'Mean':[[]]*15,'Best':[[]]*15,'Worst':[[]]*15,'SD':[[]]*15})
    function = 8 ######################
    parameval = 'Mmax'######################
    c1 = 2
    c2 = 0
    
    #order: fdmin fdmax dseta Mmax c2 lambd Smax Smin n 
    dseta = np.array([1e-5,1e-4,1e-3,1e-2,5e-2,1e-1,5e-1])
    fdmax = np.array([1e-200,1e-100,1e-50,1e-25,1e-10,1e-5,1e-1,1])
    fdmin = np.array([1e-200,1e-100,1e-50,1e-25,1e-10,1e-5,1e-1,1])
    Mmax = np.arrray([0.0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0,1.1,1.2,1.3])
    lambd = np.arrray([0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0,1.1,1.2,1.3])
    Smax = np.arrray([0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0])
    Smin = np.arrray([0.01,0.02,0.03,0.04,0.05,0.06,0.07,0.08,0.09,0.1])
    n = np.arange([1,5,10,20,30,40,50,60,70,80,90,100])
    c2 = np.array([0.0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0,1.1,1.2,1.3,1.5,2.0])
    #c3 = np.array([0.2,0.4,0.6,0.8,1.0,1.2,1.4])

    #limrestart = np.arange([20,40,60,80,100,120])
    
    dseta2 = dseta[5] # 0.001
    fdmax2 = fdmax[5] # 1e-25
    fdmin2 = fdmin[5] # 1e-25
    Mmax2 = Mmax[5]   # 0.7
    lambd2 = lambd[5] # 0.7
    Smax2 = Smax[5]   # 0.5
    Smin2 = Smin[5]   # 0.05
    n2 = n[5]         # 50
    #c2 1.0
  
    dseta2 = 1e-1
    fdmax2 = 1e-6
    fdmin2 = 1e-6
    n2 = 40

    
    fevalmax = 3e6
    runs = 5
    np.random.seed(0)
    tt = TicToc()
    Paramets ={'Parameter':[],'Mean Cost':[]}
    config = []
    load = False  ##########


    i= 0
    while i < (eval(parameval+'.shape[0]')):
        run_list = []
        BEST1_2_5 = []
        BEST6_0_5 = []
        BEST3_0_6 = []
        exec(parameval+'2'+'='+parameval+'[i]')
        j = 0
        while j < runs:
            if load:
                dill.load_session('./tmp/'+parameval+'_'+'saveworkspace.pkl')
                load = False
                i-=1
                j-=1
                itmax = int(3e6/n2)
                    
            tt.tic()
            
            bench = Benchmark()
            info = bench.get_info(function)
            f = bench.get_function(function)
            D = info['dimension']
            Lu = info['upper']
            Ll = info['lower']


            GBlist,GB1,X1 = DSRegPSO(D,n2,Ll,Lu,c1,c2,Mmax2,lambd2,fdmax2,fdmin2,
                            Smax2,Smin2,dseta2,fevalmax, j+1, {parameval:eval(parameval+'[i]')},function,f)
            del bench, info, f

            BEST1_2_5.append(GBlist[int(1.2e5/n2)-1])
            BEST6_0_5.append(GBlist[int(6.0e5/n2)-1])

            BEST3_0_6.append(GB1)
            dill.dump_session('./tmp/'+parameval+'_'+'saveworkspace.pkl')
            tt.toc()
            j+=1
            run_list.append(GB1)
        mean_runs = np.mean(run_list)
        DATA2.loc[function-1,'Best'] = BEST3_0_6
        DATA2.loc[function-1,'Median'] = [np.median(BEST1_2_5),np.median(BEST6_0_5),np.median(BEST3_0_6)]
        DATA2.loc[function-1,'Mean'] = [np.mean(BEST1_2_5),np.mean(BEST6_0_5),np.mean(BEST3_0_6)]
        DATA2.loc[function-1,'Best'] = [np.min(BEST1_2_5),np.min(BEST6_0_5),np.min(BEST3_0_6)]
        DATA2.loc[function-1,'Worst'] = [np.max(BEST1_2_5),np.max(BEST6_0_5),np.max(BEST3_0_6)]
        DATA2.loc[function-1,'SD'] = [np.std(BEST1_2_5),np.std(BEST6_0_5),np.std(BEST3_0_6)]
        Paramets['Parameter'].append(eval(parameval+'2'))
        Paramets['Mean Cost'].append(mean_runs)
        i+=1
            
    ParamResults  =pd.DataFrame(Paramets)
    
    
    

Function 8 Iteration 7.4000E+04 Function Evaluations 2.9600E+06 Best Cost 1.5923E+14 Time 10.401067600003444
Run 5 Parameter {'Mmax': 1.2000000000000002}
n 40
dseta 0.1
fdmax 1e-06
fdmin 1e-06
LSu 92.42381806017163
delta 8.403960537100109e-05
deltamin 3.9867785676095204e-08
deltamax 9.999989124669041e-05
Ss 0.42000000000000004
Elapsed time is 793.472745 seconds.


In [15]:
ParamResults.sort_values(by = ['Mean Cost'])

Parameter     Mean Cost
7         0.7  2.690935e+13
9         0.9  3.710840e+13
10        1.0  3.849602e+13
8         0.8  4.112899e+13
6         0.6  4.506134e+13
4         0.4  4.655240e+13
3         0.3  5.186356e+13
5         0.5  5.277921e+13
2         0.2  5.420600e+13
0         0.0  9.716256e+13
1         0.1  1.077097e+14
11        1.1  1.240118e+14
12        1.2  5.882686e+14

In [16]:
DATA2

Function                                             Median  \
0        f1                                                 []   
1        f2                                                 []   
2        f3                                                 []   
3        f4                                                 []   
4        f5                                                 []   
5        f6                                                 []   
6        f7                                                 []   
7        f8  [892668423584229.4, 242831113273386.12, 150646...   
8        f9                                                 []   
9       f10                                                 []   
10      f11                                                 []   
11      f12                                                 []   
12      f13                                                 []   
13      f14                                                 []   
14      f15                                                 []   

                                                 Mean  \
0                                                  []   
1                                                  []   
2                                                  []   
3                                                  []   
4                                                  []   
5                                                  []   
6                                                  []   
7   [2084420348397131.2, 1430529980542861.2, 58826...   
8                                                  []   
9                                                  []   
10                                                 []   
11                                                 []   
12                                                 []   
13                                                 []   
14                                                 []   

                                                 Best  \
0                                                  []   
1                                                  []   
2                                                  []   
3                                                  []   
4                                                  []   
5                                                  []   
6                                                  []   
7   [557280012846514.7, 156357585731903.84, 258258...   
8                                                  []   
9                                                  []   
10                                                 []   
11                                                 []   
12                                                 []   
13                                                 []   
14                                                 []   

                                                Worst  \
0                                                  []   
1                                                  []   
2                                                  []   
3                                                  []   
4                                                  []   
5                                                  []   
6                                                  []   
7   [6866747053448693.0, 6057871478001802.0, 25566...   
8                                                  []   
9                                                  []   
10                                                 []   
11                                                 []   
12                                                 []   
13                                                 []   
14                                                 []   

                                                   SD  
0                                                  []  
1                                                  []  
2                                      

In [17]:
DATA

alg  milestone   f1   f2   f3   f4   f5   f6   f7   f8   f9  f10  \
0  DSRegPSOP   120000.0  inf  inf  inf  inf  inf  inf  inf  inf  inf  inf   
1  DSRegPSOP   600000.0  inf  inf  inf  inf  inf  inf  inf  inf  inf  inf   
2  DSRegPSOP  3000000.0  inf  inf  inf  inf  inf  inf  inf  inf  inf  inf   

   f11  f12  f13  f14  f15  
0  inf  inf  inf  inf  inf  
1  inf  inf  inf  inf  inf  
2  inf  inf  inf  inf  inf

In [18]:
DATA.loc[0,'f'+str(function)] = DATA2['Best'][function-1][0]
DATA.loc[1,'f'+str(function)] = DATA2['Best'][function-1][1]
DATA.loc[2,'f'+str(function)] = DATA2['Best'][function-1][2]


In [19]:
DATA

alg  milestone   f1   f2   f3   f4   f5   f6   f7            f8   f9  \
0  DSRegPSOP   120000.0  inf  inf  inf  inf  inf  inf  inf  5.572800e+14  inf   
1  DSRegPSOP   600000.0  inf  inf  inf  inf  inf  inf  inf  1.563576e+14  inf   
2  DSRegPSOP  3000000.0  inf  inf  inf  inf  inf  inf  inf  2.582588e+13  inf   

   f10  f11  f12  f13  f14  f15  
0  inf  inf  inf  inf  inf  inf  
1  inf  inf  inf  inf  inf  inf  
2  inf  inf  inf  inf  inf  inf

In [20]:
ParamResults = ParamResults.sort_values(by = ['Mean Cost'])

In [21]:
ParamResults.to_csv(' parameters f'+str(function) +' '+parameval+'.csv')

In [22]:
dill.dump_session('./tmp/'+'f'+str(function)+' '+parameval+'_'+'saveworkspace.pkl')